In [1]:
# !pip install autogen
# !pip install openai

# or pip install -r requirements.txt (from the github repo)

In [1]:
# source: https://github.com/microsoft/autogen/blob/main/notebook/autobuild_agent_library.ipynb
config_file_or_env = './OAI_CONFIG_LIST'
default_llm_config = {
    'temperature': 0
}

In [2]:
# Create AgentBuilder instance using config path & default config
# Specify  builder & agent models
from autogen.agentchat.contrib.agent_builder import AgentBuilder
builder = AgentBuilder(
    config_file_or_env=config_file_or_env, 
    builder_model='gpt-4o')

/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Specify the building tasks
# define building taskwith description *& examples
# helps bild manager decide on agents
# example task: "Create a multi-agent system for task X"

building_task = """Create a multi-agent system for the task of finding papers on arxiv about artificial intelligence, and
analyse its application in some domain. For example, find a latest paper about gpt-4o on arxiv and find its potential applications in software."""

In [4]:
# use build() method to generate agents
# include a user proxy for tasks involving coding

agent_list, agent_configs = builder.build(building_task, default_llm_config, coding=True)

==> Generating agents...
['Arxiv_Research_Expert', 'AI_Research_Expert', 'NLP_Expert', 'Data_Analysis_Expert', 'Domain_Application_Expert'] are generated.
==> Generating system message...
Preparing system message for Arxiv_Research_Expert
Preparing system message for AI_Research_Expert
Preparing system message for NLP_Expert
Preparing system message for Data_Analysis_Expert
Preparing system message for Domain_Application_Expert
==> Generating description...
Preparing description for Arxiv_Research_Expert
Preparing description for AI_Research_Expert
Preparing description for NLP_Expert
Preparing description for Data_Analysis_Expert
Preparing description for Domain_Application_Expert
==> Creating agents...
Creating agent Arxiv_Research_Expert...
Creating agent AI_Research_Expert...
Creating agent NLP_Expert...
Creating agent Data_Analysis_Expert...
Creating agent Domain_Application_Expert...
Adding user console proxy...


In [5]:
agent_list

In [6]:
agent_configs

{'building_task': 'Create a multi-agent system for the task of finding papers on arxiv about artificial intelligence, and\nanalyse its application in some domain. For example, find a latest paper about gpt-4o on arxiv and find its potential applications in software.',
 'agent_configs': [{'name': 'Arxiv_Research_Expert',
   'model': [],
   'tags': [],
   'system_message': '## Your role\nArxiv_Research_Expert is an expert in navigating and extracting valuable information from the arXiv repository, specifically focusing on papers related to artificial intelligence (AI). This expert possesses extensive knowledge in AI subfields and is adept at analyzing and understanding recent research developments. Arxiv_Research_Expert is proficient in identifying significant contributions, summarizing findings, and evaluating the potential applications of AI research in various domains.\n\n## Task and skill instructions\n- **Task Description**: The primary task for Arxiv_Research_Expert is to create a 

In [7]:
# Execute the task
# agents collaborate in a group chat to complete task
# combine llms, human inputs, and tools
# example: start_task(execution_task, agent_list, llm_config)
from autogen import config_list_from_json, GroupChat, GroupChatManager

config_list = config_list_from_json(config_file_or_env)

def start_task(execution_task: str, agent_list: list):
    group_chat = GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list, **default_llm_config})
    agent_list[0].initiate_chat(manager, message=execution_task)


start_task(
    execution_task="Find a recent paper about gpt-4 on arxiv.",
    agent_list=agent_list,
)

Arxiv_Research_Expert (to chat_manager):

Find a recent paper about gpt-4 on arxiv.

--------------------------------------------------------------------------------

Next speaker: Computer_terminal

Computer_terminal (to chat_manager):

There is no code from the last 1 message for me to execute. Group chat manager should let other participants to continue the conversation. If the group chat manager want to end the conversation, you should let other participant reply me only with "TERMINATE"

--------------------------------------------------------------------------------

Next speaker: Arxiv_Research_Expert

Arxiv_Research_Expert (to chat_manager):

Let's start by searching for the latest papers on GPT-4 on arXiv. I will write a Python script to query the arXiv API for papers related to GPT-4.

```python
import feedparser

def search_arxiv(query, max_results=5):
    base_url = 'http://export.arxiv.org/api/query?'
    search_query = f'search_query=all:{query}&start=0&max_results={max_r

In [8]:
builder.clear_all_agents()

All agents have been cleared.


In [9]:
saved_path = builder.save()

Building config saved to ./save_config_278c3bde745d76536b39f7f5952c1d5a.json


Configurations will be saved in JSON format like such:

```
// FILENAME: save_config_TASK_MD5.json
{
"building_task": "Find a paper on arxiv about artificial intelligence, 
and analyze its application in some domain. For example, find a latest 
paper about gpt-4 on arxiv and find its potential applications in 
software." ,
"agent_configs": [
{
    "name": "...",
    .....
},
...
],
"manager_system_message": "...",
"code_execution_config": {...},
"default_llm_config": {...}
}

```